In [8]:
import fitz  # PyMuPDF


In [9]:
import camelot
import pandas as pd

In [10]:
import json
from pathlib import Path

In [11]:
pdf_path = Path("../data/raw_pdfs/TCS_2023-24.pdf")
if not pdf_path.exists(): raise FileNotFoundError(f"PDF file not found: {pdf_path}")

In [28]:
from pypdf import PdfReader
from pathlib import Path
import re
import json

pdf_path = Path("../data/raw_pdfs/TCS_2023-24.pdf")

reader = PdfReader(str(pdf_path))
all_rows = []
row_counter = 0

date_cols = []
year_cols = []

# ---- Map (Date + Year) → Business Label ----
def map_to_label(date, year):
    # Standardize FY logic
    if date.startswith("March 31"):
        return f"Q4 FY{year[-2:]}"   # last 2 digits of year
    elif date.startswith("December 31"):
        return f"Q3 FY{year[-2:]}"
    else:
        return f"FY{year[-2:]}"      # fallback annual

for page_num, page in enumerate(reader.pages, start=1):
    text = page.extract_text()
    if not text:
        continue

    lines = text.split("\n")
    for line in lines:
        line = line.strip()
        if not line:
            continue

        row_counter += 1

        # Row 1: dates
        if row_counter == 1:
            date_cols = [d.strip() for d in line.split(",") if d.strip()]
            continue

        # Row 2: years
        if row_counter == 2:
            year_cols = line.split()
            continue

        # Row 3: build merged headers
        if row_counter == 3 and date_cols and year_cols:
            merged_headers = []
            for i in range(min(len(date_cols), len(year_cols))):
                merged_headers.append(map_to_label(date_cols[i], year_cols[i]))
            merged_headers = merged_headers[:5] + [""] * (5 - len(merged_headers))
            all_rows.append(["KPI"] + merged_headers)


        # Try normal 2+ space split
        parts = re.split(r"\s{2,}", line)

        if len(parts) == 1:
            tokens = line.split()
            if len(tokens) >= 6:
                kpi = " ".join(tokens[:-5])
                values = tokens[-5:]
                all_rows.append([kpi] + values)
            else:
                all_rows.append([line] + [""] * 5)

        elif len(parts) == 2:
            kpi = parts[0].strip()
            values = parts[1].split()
            values = values[:5] + [""] * (5 - len(values))
            all_rows.append([kpi] + values)

        else:
            kpi = parts[0].strip()
            values = " ".join(parts[1:]).split()
            values = values[:5] + [""] * (5 - len(values))
            all_rows.append([kpi] + values)

# Save JSON
result_json = {"rows": all_rows}
with open("financial_statement_fixed_2024.json", "w", encoding="utf-8") as f:
    json.dump(result_json, f, ensure_ascii=False, indent=2)

# Preview
for r in all_rows[:10]:
    print(r)


['KPI', 'Q4 FY24', 'Q3 FY23', 'Q4 FY23', 'Q4 FY24', 'Q4 FY23']
['Revenue from operations', '61,237', '60,583', '59,162', '2,40,893', '2,25,458']
['Other income', '1,157', '862', '1,175', '4,422', '3,449']
['TOTAL INCOME', '62,394', '61,445', '60,337', '2,45,315', '2,28,907']
['Expenses', '', '', '', '', '']
['Employee benefit expens es', '35,138', '34,722', '33,687', '1,40,131', '1,27,522']
['Cos t of equipment and s oftware licences', '1,561', '1,173', '620', '3,702', '1,881']
['Finance cos ts', '226', '230', '272', '778', '779']
['Depreciation and amortis ation expens e', '1,246', '1,233', '1,286', '4,985', '5,022']
['Other expens es', '7,374', '8,300', '9,081', '32,764', '36,796']


In [29]:
import pandas as pd
import json

# Load JSON with cleaned rows
with open("financial_statement_fixed_2024.json", "r", encoding="utf-8") as f:
    data = json.load(f)

rows = data["rows"]

# Convert JSON rows to DataFrame
df = pd.DataFrame(rows)

# The first row in JSON already contains our clean headers like:
# ["KPI", "Q4 FY24", "Q3 FY24", "Q4 FY23", "FY24", "FY23"]
headers = df.iloc[0].tolist()
df.columns = headers

# Drop that header row
df = df.drop(0).reset_index(drop=True)

# Rename first col to KPI (in case it's not already)
df.rename(columns={df.columns[0]: "KPI"}, inplace=True)

# Replace NaN with blanks
df = df.fillna("")

# Show nicely in Jupyter
from IPython.display import display
pd.set_option('display.max_colwidth', None)
display(df.head(25))


,KPI,Q4 FY24,Q3 FY23,Q4 FY23,Q4 FY24,Q4 FY23
0,Revenue from operations,"61,237","60,583","59,162","2,40,893","2,25,458"
1,Other income,"1,157",862,"1,175","4,422","3,449"
2,TOTAL INCOME,"62,394","61,445","60,337","2,45,315","2,28,907"
3,Expenses,,,,,
4,Employee benefit expens es,"35,138","34,722","33,687","1,40,131","1,27,522"
5,Cos t of equipment and s oftware licences,"1,561","1,173",620,"3,702","1,881"
6,Finance cos ts,226,230,272,778,779
7,Depreciation and amortis ation expens e,"1,246","1,233","1,286","4,985","5,022"
8,Other expens es,"7,374","8,300","9,081","32,764","36,796"
9,TOTAL EXPENSES,"45,545","45,658","44,946","1,82,360","1,72,000"


In [30]:
from pypdf import PdfReader
from pathlib import Path
import re
import json

pdf_path = Path("../data/raw_pdfs/TCS_2024-25.pdf")

reader = PdfReader(str(pdf_path))
all_rows = []
row_counter = 0

date_cols = []
year_cols = []

# ---- Map (Date + Year) → Business Label ----
def map_to_label(date, year):
    # Standardize FY logic
    if date.startswith("March 31"):
        return f"Q4 FY{year[-2:]}"   # last 2 digits of year
    elif date.startswith("December 31"):
        return f"Q3 FY{year[-2:]}"
    else:
        return f"FY{year[-2:]}"      # fallback annual

for page_num, page in enumerate(reader.pages, start=1):
    text = page.extract_text()
    if not text:
        continue

    lines = text.split("\n")
    for line in lines:
        line = line.strip()
        if not line:
            continue

        row_counter += 1

        # Row 1: dates
        if row_counter == 1:
            date_cols = [d.strip() for d in line.split(",") if d.strip()]
            continue

        # Row 2: years
        if row_counter == 2:
            year_cols = line.split()
            continue

        # Row 3: build merged headers
        if row_counter == 3 and date_cols and year_cols:
            merged_headers = []
            for i in range(min(len(date_cols), len(year_cols))):
                merged_headers.append(map_to_label(date_cols[i], year_cols[i]))
            merged_headers = merged_headers[:5] + [""] * (5 - len(merged_headers))
            all_rows.append(["KPI"] + merged_headers)


        # Try normal 2+ space split
        parts = re.split(r"\s{2,}", line)

        if len(parts) == 1:
            tokens = line.split()
            if len(tokens) >= 6:
                kpi = " ".join(tokens[:-5])
                values = tokens[-5:]
                all_rows.append([kpi] + values)
            else:
                all_rows.append([line] + [""] * 5)

        elif len(parts) == 2:
            kpi = parts[0].strip()
            values = parts[1].split()
            values = values[:5] + [""] * (5 - len(values))
            all_rows.append([kpi] + values)

        else:
            kpi = parts[0].strip()
            values = " ".join(parts[1:]).split()
            values = values[:5] + [""] * (5 - len(values))
            all_rows.append([kpi] + values)

# Save JSON
result_json = {"rows": all_rows}
with open("financial_statement_fixed_2025.json", "w", encoding="utf-8") as f:
    json.dump(result_json, f, ensure_ascii=False, indent=2)

# Preview
for r in all_rows[:10]:
    print(r)


['KPI', 'Q4 FY25', 'Q3 FY24', 'Q4 FY24', 'Q4 FY25', 'Q4 FY24']
['Revenue from operations', '64,479', '63,973', '61,237', '2,55,324', '2,40,893']
['Other income', '1,028', '1,243', '1,157', '3,962', '4,422']
['TOTAL INCOME', '65,507', '65,216', '62,394', '2,59,286', '2,45,315']
['Expenses', '', '', '', '', '']
['Employee benefit expenses', '36,762', '35,956', '35,138', '1,45,788', '1,40,131']
['Cost of equipment and software licences', '2,748', '3,519', '1,561', '11,648', '3,702']
['Finance costs', '227', '234', '226', '796', '778']
['Depreciation and amortisation expense', '1,379', '1,377', '1,246', '5,242', '4,985']
['Other expenses', '7,989', '7,464', '7,374', '30,481', '32,764']


In [32]:
import pandas as pd
import json

# Load JSON with cleaned rows
with open("financial_statement_fixed_2025.json", "r", encoding="utf-8") as f:
    data = json.load(f)

rows = data["rows"]

# Convert JSON rows to DataFrame
df = pd.DataFrame(rows)

# The first row in JSON already contains our clean headers like:
# ["KPI", "Q4 FY24", "Q3 FY24", "Q4 FY23", "FY24", "FY23"]
headers = df.iloc[0].tolist()
df.columns = headers

# Drop that header row
df = df.drop(0).reset_index(drop=True)

# Rename first col to KPI (in case it's not already)
df.rename(columns={df.columns[0]: "KPI"}, inplace=True)

# Replace NaN with blanks
df = df.fillna("")

# Show nicely in Jupyter
from IPython.display import display
pd.set_option('display.max_colwidth', None)
display(df.head(25))


,KPI,Q4 FY25,Q3 FY24,Q4 FY24,Q4 FY25,Q4 FY24
0,Revenue from operations,"64,479","63,973","61,237","2,55,324","2,40,893"
1,Other income,"1,028","1,243","1,157","3,962","4,422"
2,TOTAL INCOME,"65,507","65,216","62,394","2,59,286","2,45,315"
3,Expenses,,,,,
4,Employee benefit expenses,"36,762","35,956","35,138","1,45,788","1,40,131"
5,Cost of equipment and software licences,"2,748","3,519","1,561","11,648","3,702"
6,Finance costs,227,234,226,796,778
7,Depreciation and amortisation expense,"1,379","1,377","1,246","5,242","4,985"
8,Other expenses,"7,989","7,464","7,374","30,481","32,764"
9,TOTAL EXPENSES,"49,105","48,550","45,545","1,93,955","1,82,360"
